In [13]:
import pprint
import os
import io
import nltk
from nltk.corpus import stopwords
from urlextract import URLExtract
import re
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities

In [14]:
article_corpus = []
listnames = os.listdir("Articlestxt/")
urlpattern =r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

In [15]:
for i in range(len(listnames)):
    with io.open('Articlestxt/'+listnames[i],'r',encoding="utf-8") as f:
        article_corpus.append(f.read() )
        article_corpus[i]=re.sub(urlpattern,'',article_corpus[i])
        article_corpus[i]=re.sub('\(*\d+\)','',article_corpus[i])

In [16]:
stoplist = set(stopwords.words('english'))
texts = [[word for word in document.lower().split() if word not in stoplist] for document in article_corpus]

In [17]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
processed_corpus = [[token for token in text if frequency[token] > 2] for text in texts]

In [18]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(29222 unique tokens: ['&', '(30min)', '(amit', '(an', '(barratt']...)


In [19]:
##new_doc = "Human computer interaction"
##new_vec = dictionary.doc2bow(new_doc.lower().split())
##print(new_vec)

In [20]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [21]:
tfidf = models.TfidfModel(bow_corpus)

In [22]:
index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(dictionary)-1)

In [32]:
with io.open('lol.txt','r',encoding='utf-8') as f:
    querydoc = f.read().split()
    
    
query_bow = dictionary.doc2bow(querydoc)
sims = index[tfidf[query_bow]]


In [33]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(listnames[document_number][:-4]+" --------->", score)

Very important-BC-systematic ---------> 0.064931534
1-s2.0-S0019850120308518-main ---------> 0.051644526
sd.2088 ---------> 0.03933922
Ex of new tech and sustain ---------> 0.037517384
1-s2.0-S0040162520313081-main ---------> 0.032709707
jiec.12051 ---------> 0.03242848
bse.2684 ---------> 0.030632395
impact of digital tech on eco_ env perform ---------> 0.029765787
Blockchain and carbon emissions ---------> 0.027860083
Industry 4.0_ logistics ---------> 0.027636837
tran.12363 ---------> 0.027065115
beer.12259 ---------> 0.026932208
IoT _ SSC ---------> 0.025978561
1-s2.0-S2096248717300073-main ---------> 0.02538016
j.1530-9290.2012.00512.x ---------> 0.023234999
1-s2.0-S2351978920316541-main ---------> 0.022907395
Very import(include references) ---------> 0.022456486
jsc.2361 ---------> 0.020602023
1-s2.0-S259012302030058X-main ---------> 0.020442784
1-s2.0-S2352864819301166-main ---------> 0.020247297
digitalization _ sust in FMCG ---------> 0.019446276
agri-food _ SC_ technologies 